In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds

# DATA COLLECTION
if os.path.exists("movies.csv") and os.path.exists("ratings.csv"):
    movies = pd.read_csv("movies.csv")
    ratings = pd.read_csv("ratings.csv")
    print("Step 1 : Data loaded.")
    
    # TRAIN-TEST SPLIT 
    train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)
    print(" Step 2 : Data split into train and test sets.")
else:
    print("Files not found. Please upload the files ")

    # DATA PROCESSING
    # user-item Interaction Matrix
user_item_matrix = train_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# MODEL BUILDING & TUNING (SVD)
u, sigma, vt = svds(user_item_matrix.values, k=50)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(u, sigma), vt)

# MODEL TRAINING
predicted_df = pd.DataFrame(all_user_predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index)
print(" step 3 : Matrix factorization complete.")

print("\t")

def recommend_movies(user_id, num_recommendations=10):
    
    user_predictions = predicted_df.loc[user_id].sort_values(ascending=False)
    user_rated_movies = ratings[ratings['userId'] == user_id]['movieId']
    
    recommendations = user_predictions[~user_predictions.index.isin(user_rated_movies)]
    
    top_recs = movies[movies['movieId'].isin(recommendations.head(num_recommendations).index)].copy()
    top_recs['predicted_rating'] = recommendations.head(num_recommendations).values
    
    return top_recs[['movieId', 'title', 'predicted_rating']]


user_id_to_recommend = 10
results = recommend_movies(user_id_to_recommend)
print(f"Top 10 Recommendations for User {user_id_to_recommend}:")
print(results)
print("\t")
print(" Here is your top 10 recommendation.Enjoy the movie. ")

Step 1 : Data loaded.
 Step 2 : Data split into train and test sets.
 step 3 : Matrix factorization complete.
	
Top 10 Recommendations for User 10:
      movieId                                              title  \
277       318                   Shawshank Redemption, The (1994)   
3141     4226                                     Memento (2000)   
3568     4886                              Monsters, Inc. (2001)   
4076     5816     Harry Potter and the Chamber of Secrets (2002)   
4427     6539  Pirates of the Caribbean: The Curse of the Bla...   
5166     8368    Harry Potter and the Prisoner of Azkaban (2004)   
6405    50872                                 Ratatouille (2007)   
6885    63082                         Slumdog Millionaire (2008)   
7302    76093                    How to Train Your Dragon (2010)   
7372    79132                                   Inception (2010)   

      predicted_rating  
277           1.874137  
3141          1.357853  
3568          1.294241  
407